<a href="https://colab.research.google.com/github/ucfilho/Codigos_Teste/blob/master/ABCOptim_mar_11_2019_AINDA_BUGADO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
Artificial bee colony optimization 
https://abc.erciyes.edu.tr/pub/Step%20by%20Step%20Procedure%20of%20ABC.pdf
'''

'\nArtificial bee colony optimization \nhttps://abc.erciyes.edu.tr/pub/Step%20by%20Step%20Procedure%20of%20ABC.pdf\n'

In [0]:
import numpy as np
from random import randint

In [0]:
'''Rosembrock Function'''
def Fun(x):
    fun=0
    a=1.0
    b=100.0
    fun = (a-x[0])**2 + b*(x[1]-x[0]**2)**2
    return fun

In [0]:
''' 
FOBJ gera vetorialmente os valores com a funcao objetivo escalar
em outras palavras gera para populacao o valor da funcao 
a ser otimizada
'''
def FOBJ(X):
  X=np.matrix(X)
  rows = X.shape[0]
  fobj=np.zeros(rows)
  for i in range(rows):
    fobj[i]=Fun(X[i,])
  return fobj

In [0]:
'''roulette wheel selection python'''
'''function i=RouletteWheelSelection(P)
    r=rand;
    
    C=cumsum(P);
    
    i=find(r<=C,1,'first');
end'''
def RouletteWheelSelection(P):
  index=-1
  r=np.random.random()
  C=np.cumsum(P)
  for  item in C:
    index=index+1
    if (item >= r):
      break
  return index

In [0]:
'''Enxame retorna a populacao aleatoria com todos valores entre MIN e MAX'''
def Enxame(PAR,NPAR,MAX,MIN):
    x=np.zeros((NPAR, len(MAX)))
    for j in range(len(MAX)):
        for i in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

In [0]:
''' 
Fitness gera vetorialmente os valores com a funcao fitness
equivale a quantidade de nectar da fonte
'''
def FIT(X):
    rows = X.shape[0]
    fit=np.zeros(rows)
    fit=np.copy(FOBJ(X))
    for i in range(rows):
        if(fit[i]>=0):
            fit[i]=1/(fit[i]+1)
        else:
            fit[i]=1+abs(fit[i])
    return fit

In [0]:
'''
probablilidade usada para roleta q atualizacao das onlookers 
semelhante ao genetico
'''

def PROB(x):
    rows=x.shape[0]
    fit=np.zeros(rows)
    prob=np.zeros(rows)
    fit=FIT(x)
    soma=np.sum(fit)
    
    for i in range(rows):
        prob[i] = fit[i]/soma

    return prob

In [0]:
'''
Obtem a melhor escolha greedy search
'''
def GetBest(x,xbest):
    ycal=FOBJ(x)
    best=np.argmin(ycal)
    yref=Fun(xbest)
    if(yref<ycal[best]):
        GBEST=xbest
    else:
        GBEST=x[best,]
    return GBEST
    

In [0]:
'''
employed bee phase
'''
def EmployedBee(xo,x,MAX,MIN) :
    NPAR = xo.shape[0]
    PAR= xo.shape[1]
    V=np.zeros((NPAR, len(MAX)))

    for j in range(PAR):
        for i in range(NPAR):           
            fi=np.random.uniform(low=-2.0, high=2.0, size=None)
            rd=randint(0, (NPAR-1))
            V[i,j]=xo[i,j]+(xo[i,j] -x[rd,j])*fi # candidata a solucao
            if (V[i,j]>MAX[j]):
              V[i,j]=np.copy(MAX[j]) # restringe a busca para o intervalo
            if (V[i,j]<MIN[j]):
              V[i,j]=np.copy(MIN[j]) # restringe a busca para o intervalo
              
    YCAL=FOBJ(xo)
    YV=FOBJ(V)
    for i in range(NPAR):
        if(YV[i]<YCAL[i]):
            x[i,]=np.copy(V[i,])

    return x

In [0]:
'''
onlooker bee phase
'''
def OnlookerBee(xo,x,trial,ntrial,MAX,MIN) : #OnlookerBee(xo,x_Employed,...)
    rows = xo.shape[0]
    cols = xo.shape[1]
    NPAR = xo.shape[0]
    PAR= xo.shape[1]
    prob=PROB(x)
    prob_0=PROB(xo)    
    for i in range(rows):
      rd=RouletteWheelSelection(prob_0)
      fi=np.random.uniform(low=-1.0, high=1.0, size=None)
      Xmi=xo[i,]+fi*(xo[i,]-x[rd,])
      if(Fun(xo[i,])>Fun(Xmi)):
        xo[i,]=np.copy(Xmi)
        trial[i]=0
      else:
        trial[i]=trial[i]+1

    return xo,trial

In [0]:
'''
scout bee phase
'''
#def ScoutBee(trial,x,ntrail,MAX,MIN):
def ScoutBee(x,MAX,MIN,trail,ntrail):
    PAR = x.shape[0]
    NPAR = x.shape[1]
    x_scout=[]
    x_scout=x.copy
    
    for i in range(PAR):
      if(trail[i]>=ntrail): #scout abandona a fonte
        trail[i]=0
        for j in range(NPAR):
          x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()

    return x,trial

In [0]:

'''
Metodo que alterna e coordena o uso employers,onlooker, scout bees
e greedy search

'''
def ABCOPtim(ITE,PAR,ntrial,NPAR,MAX,MIN):

    trial=np.ones(NPAR) # inicializa contador p abandonar fonte de alimento
    xbest=Enxame(PAR,1,MAX,MIN)[0,] #inicializa xbest
 
    vbest_old=Fun(xbest)
    vbest_new=Fun(xbest)
    x=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee-parte 1
    xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee-parte 1
    Fo=FOBJ(xo) # vai variar Employed bee por ntrial aqui p iniciar
    Fnew=FOBJ(x) # vai variar Employed bee por ntrial aqui p iniciar
    
    for i in range(ITE):
      
      for k in range(NPAR):
        if(Fo[k] <= Fnew[k]):
          trial[k]=trial[k]+1
        else:
          trial[k]=0
          xo[k,]=np.copy(x[k,])
      
      x_Employed =EmployedBee(xo,x,MAX,MIN)
          
      x=[]
      x=x_Employed.copy()
      xbest=GetBest(x_Employed,xbest)
      vbest_new=Fun(xbest)
      xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa OnlookerBee
      x_Onlooker=OnlookerBee(xo,x_Employed,trial,ntrial,MAX,MIN)
      xbest=GetBest(x_Onlooker,xbest)
      x_Scout,trail=ScoutBee(x,MAX,MIN,trail,ntrail) # inicializa ScoutBee ( a rigor so esta linha basta)
      xbest=GetBest(x_Scout,xbest)
      
      return xbest, x_Onlooker,x_Employed          
      
  

     

In [14]:
NPAR=500 #PARTICULAS
ITE=3000 #ITERACOES
PAR=2 #NUM DE PARAMETROS A SER OTIMIZADOS
MAX=[2,2] # MAXIMO DE CADA PARAMETRO
MIN=[-2,-2] # MINIMO DE CADA PARAMETRO
ntrail=10 #numero de buscas ate abandonar uma fonte de alimento

print(ntrail)

for i in range(6):
    print("resolucao",i+1," ")
    xbest, x_Onlooker,x_Employed =ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN)
    print("vetor",xbest,"funcao", Fun(xbest),"\n")

'''print(xbest)
print(x_Onlooker)
print(x_Employed)'''
    

10
resolucao 1  


ValueError: ignored